<a href="https://colab.research.google.com/github/rdemarqui/topic_modeling/blob/main/customer_claim_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer claims analysis

## Open Dataset

In [1]:
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore', 'error')

In [2]:
import pandas as pd

In [3]:
data_url = 'https://raw.githubusercontent.com/rdemarqui/topic_modeling/main/datasets/reclame_aqui_full.csv'
df = pd.read_csv(data_url)
print(df.shape)
df.head()

(500, 5)


,Unnamed: 0.1,Unnamed: 0,Title,link,description
0,0,0,Claro residência,/claro/claro-residencia_h-VgVeL0ez_dO42I/,Estou há mais de 7 dias tentando efetuar a con...
1,1,1,NO CONSIGO FALAR COM UM HUMANO,/claro/no-consigo-falar-com-um-humano_0kiSV_xg...,Realizei o pagamento das faturas atrasadas e s...
2,2,2,Oi virou claro e não recebe ligações!!!,/claro/oi-virou-claro-e-nao-recebe-ligacoes_MG...,Minha mãe tinha um chip da Oi que virou Claro ...
3,3,3,Mudança de plano sem a minha autorização,/claro/mudanca-de-plano-sem-a-minha-autorizaca...,Na sexta-feira dia 6 de outubro fiz uma recarg...
4,4,4,Plano PREZAO,/claro/plano-prezao_iW3NARVJ71tk_jVM/,"Bom diaNa data de 08/10/2023, recebi um SMS in..."


## Text Preprocess

The dataset that we will use was acquired through web scrapping on reclameaqui.com.br. This data is messy, since the users can write everithing they want. Because of that, we'll need to clean and make some modifications bofore we create our model.

### Normalize

Firstly, we will normalize our text. This proccess goes through some steps such as removing punctuation, accentuation, special characters and masked words.

In [4]:
import string

In [5]:
punctuations = list(string.punctuation)

def remove_punctuation(text):
  return ''.join([char if char not in punctuations else ' ' for char in text])

In [6]:
accentuation = {
    "á": "a", "ã": "a", "à": "a","â": "a",
    "é": "e","ê": "e",
    "í": "i",
    "ó": "o","õ": "o", "ô":"o",
    "ú": "u",
    "ç": "c"
    }

def remove_accentuation(text):
  return ''.join(accentuation.get(char, char) for char in text)

In [7]:
special_charaters = [
    "1","2","3","4","5","6","7","8","9","0",
    " a "," b "," c "," d "," e "," f "," g "," h "," i "," j "," k "," l ",
    " m "," n "," o "," p "," q "," r "," s "," t "," u "," v "," x "," z ",
    "r$", "$",
    ]

def remove_special_characters(text):
  return ''.join([char if char not in special_charaters else ' ' for char in text])

Reclame Aqui website often masks data when it is sensitive information or offensive words using "editado por reclame aqui" in place. We will remove it from our database.

In [8]:
def remove_mask(text):
  return text.replace("editado pelo reclame aqui", "")

In [9]:
def normalize_text(df, text_field, lower=True, rem_punct=True, rem_accent=True, rem_spec_caract=True, rem_mask=True):
  df[text_field + "_clean"] = df[text_field].astype(str)
  if lower: df[text_field + "_clean"] = df[text_field + "_clean"].str.lower()
  if rem_punct: df[text_field + "_clean"] = df[text_field + "_clean"].apply(remove_punctuation)
  if rem_accent: df[text_field + "_clean"] = df[text_field + "_clean"].apply(remove_accentuation)
  if rem_spec_caract: df[text_field + "_clean"] = df[text_field + "_clean"].apply(remove_special_characters)
  if rem_mask: df[text_field + "_clean"] = df[text_field + "_clean"].apply(remove_mask)
  df[text_field + "_clean"] = df[text_field + "_clean"].replace(r'\s+', ' ', regex=True) #remove spaces

In [10]:
# Normalizing text in a new column
normalize_text(df, 'description')

### Stop Words

Stop words make the topics poor. Removing stop words can reduce noise and improve the efficiency of text analysis algorithms.

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
stops_nltk = nltk.corpus.stopwords.words('portuguese')
stop_words = list(stops_nltk)
stop_words = [remove_accentuation(word) for word in stop_words]
stop_words = list(set(stop_words))

def remove_stop_words(text):
    words_list = text.split()
    words_list = [word for word in words_list if word not in stop_words]
    text = ' '.join(words_list)

    return text

In [13]:
# Removing Stop Words
df["description_clean_stop"] = df["description_clean"].apply(remove_stop_words)

### Lemmatize

Lemmatization is a natural language processing technique that reduces words to their base or root form. It helps in standardizing words, improving text analysis, and simplifying language understanding.

In [14]:
import spacy.cli

language_models = ['pt_core_news_sm', 'pt_core_news_md', 'pt_core_news_lg']
spacy_model = language_models[2]

disable = ['tagger', 'parser', 'ner', 'entity_ruler', 'entity_linker', 'textcat',
           'morphologizer','attribute_ruler']

try:
  nlp = spacy.load(spacy_model, disable=disable)
except:
  spacy.cli.download(spacy_model)
  nlp = spacy.load(spacy_model, disable=disable)

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
['tok2vec', 'lemmatizer']


In [15]:
def lemma_pipe(doc_list):
  lemma_text_list = []
  for doc in nlp.pipe(doc_list, n_process=-1):
      lemma_text_list.append(" ".join(token.lemma_ for token in doc))

  return lemma_text_list

In [16]:
# Applying lemmatization
df["description_clean_stop_lemma"] = lemma_pipe(df["description_clean_stop"])

### TF-IDF

Removing low TF-IDF words is essential in topic modeling because these words probably provide little discriminatory power and can dominate topics. By excluding them, the model focuses on more meaningful and distinctive terms, improving topic quality and helping to identify relevant keywords that truly characterize the underlying topics in a corpus.

In [17]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

import gensim
import gensim.corpora as corpora
from gensim.models import TfidfModel

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [18]:
documents = df["description_clean_stop_lemma"]

In [19]:
# Tokenize text
token_docs = [word_tokenize(text) for text in documents]

# Convert text to frequency dictionary
dictionary = corpora.Dictionary(token_docs)

# Convert text in bag of words
corpus = [dictionary.doc2bow(text) for text in token_docs]

# Calculating TF-IDF
tfidf = TfidfModel(corpus, id2word=dictionary)

In [20]:
# Filtering low TF-IDF values
low_value = 0.01
low_value_words = []
for bow in corpus:
    low_value_words += [id for id, value in tfidf[bow] if value < low_value]

# Removing low TF-IDF from dictionary
dictionary.filter_tokens(bad_ids=low_value_words)

# Recompute the corpus with low value words are filtered out:
corpus = [dictionary.doc2bow(doc) for doc in token_docs]

## Latent Dirichlet Allocation (LDA)

In [21]:
# LDA implmentation
lda_model = gensim.models.LdaModel(corpus,
                                   num_topics=15,
                                   id2word=dictionary,
                                   random_state=42,
                                   update_every=1,
                                   chunksize=50,
                                   passes=15,
                                   alpha="auto")

### Visualization

In [22]:
try:
  import pyLDAvis.gensim
except:
  !pip install pyLDAvis==2.1.2
  clear_output()
  import pyLDAvis.gensim

In [23]:
warnings.filterwarnings('ignore', 'error')
vis = pyLDAvis.gensim.prepare(lda_model, corpus, lda_model.id2word, mds="mmds", R=20)
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pyLDAvis/_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  head(R).drop('saliency', 1)
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


## Bertopic

As we could see above, work with LDA demands a lot of text prepocessing. Now we will use a completely different alternative called Bertopic.

In [ ]:
device = !nvidia-smi -L
device = "cuda" if device[0][:3] == 'GPU' else "cpu"
device

In [25]:
try:
  from bertopic import BERTopic
except:
  !pip install bertopic
  clear_output()
  from bertopic import BERTopic

In [ ]:
#try:
  from sentence_transformers import SentenceTransformer
#except:
#  !pip install sentence_transformers
#  clear_output()
#  from sentence_transformers import SentenceTransformer

In [36]:
sentence_model = SentenceTransformer('rufimelo/bert-large-portuguese-cased-sts', device=device)
clear_output()
topic_model = BERTopic(embedding_model=sentence_model)

In [ ]:
topics, probs = topic_model.fit_transform(df.description_clean_stop.astype("str").tolist())

In [42]:
topic_model.visualize_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [43]:
topic_model.visualize_barchart()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [44]:
topic_model.visualize_heatmap()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
# Visualize the topic representation of major topics per class

In [ ]:
# Prepare data and classes
data = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))
docs = data["data"]
classes = [data["target_names"][i] for i in data["target"]]

# Create topic model and calculate topics per class
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)
topics_per_class = topic_model.topics_per_class(docs, classes=classes)